In [1]:
from urllib2 import Request, urlopen, URLError
import pandas as pd
import numpy as np
import os
import sqlalchemy as sql

In [2]:
f = os.path.join(os.path.expanduser("~"), "Downloads/911.csv")
df = pd.read_csv(f, low_memory=False)

In [3]:
df.head()

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,Census Tract,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,M2,8100.2001,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,07/17/2010 08:50:00 PM,36XX BLOCK OF DISCOVERY PARK BLVD,Q,Q1,5700.1012,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 08:55:00 PM,21XX BLOCK OF 3RD AVE,M,M2,7200.2025,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:00:00 PM,7XX BLOCK OF ROY ST,D,D1,7200.1002,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 09:00:00 PM,9XX BLOCK OF ALOHA ST,D,D1,6700.1009,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN


In [4]:
print "There are %i observations in this dataset" % df.shape[0]

There are 1261516 observations in this dataset


In [5]:
def time_convert(time, meridian):
    """
    """
    if isinstance(time, float):
        return np.nan
    H, M, S = time.split(":")
    if meridian == "AM":
        time2 = time
        if H == "12":
            time2 = ":".join(["00", M, S])
    else:
        if H == "12":
            time2 = time
        else:
            time2 = ":".join([str(int(H) + 12), M, S])
    return time2

def extract_time_el(time_str, el=0):
    """
    """
    if isinstance(time_str, str):
        time_el = time_str.split(" ")[el]
    else:
        time_el = np.nan
    return time_el

In [6]:
# lets only use data where there is locations and time stamps
df = df[df["Event Clearance Date"].notnull()]
df = df[df["Incident Location"].notnull()]

# replace the NaN of clearnce and initial with Uknown category 
df.loc[df["Event Clearance SubGroup"].isnull(), "Event Clearance SubGroup"] = "UNKNOWN"
df.loc[df["Initial Type Subgroup"].isnull(), "Initial Type Subgroup"] = "UNKNOWN"

In [7]:
df["date"] = df["Event Clearance Date"].apply(lambda x: extract_time_el(x, 0))
df["time12"] = df["Event Clearance Date"].apply(lambda x: extract_time_el(x, 1))
df["meridian"] = df["Event Clearance Date"].apply(lambda x: extract_time_el(x, 2))
df["time"] = df.apply(lambda x: time_convert(x["time12"], x["meridian"]), axis=1)
df["date_at_scene"] = df["At Scene Time"].apply(lambda x: extract_time_el(x, 0))
df["time12_at_scene"] = df["At Scene Time"].apply(lambda x: extract_time_el(x, 1))
df["meridian_at_scene"] = df["At Scene Time"].apply(lambda x: extract_time_el(x, 2))
df["time_at_scene"] = df.apply(lambda x: time_convert(x["time12_at_scene"], x["meridian_at_scene"]), axis=1)

In [8]:
df = df.rename(columns={"Census Tract": "census_tract", 
                        "Initial Type Subgroup": "initial_group",
                        "Event Clearance SubGroup": "clearance_group",
                        "Longitude": "longitude",
                        "Latitude": "latitude", 
                        "CAD CDW ID": "call_id"})
keep_cols = ["call_id", "initial_group", "clearance_group", "longitude", 
             "latitude", "time", "date", "date_at_scene", "time_at_scene"]
df = df[keep_cols]

# Here are the groupings of types of calls responded to

In [9]:
df["time"] = df["date"] + " " + df["time"]
df["time_at_scene"] = df["date_at_scene"] + " " + df["time_at_scene"]

In [10]:
ini_group = pd.DataFrame({"initial_group_description": df["initial_group"].unique(),
                          "initial_group_id": range(len(df["initial_group"].unique()))})
cl_group = pd.DataFrame({"clearance_group_description": df["clearance_group"].unique(),
                          "clearance_group_id": range(len(df["clearance_group"].unique()))})

In [11]:
df = df.merge(cl_group.rename(columns={"clearance_group_description": "clearance_group"}), 
              on="clearance_group", how="left")

In [12]:
df = df.merge(ini_group.rename(columns={"initial_group_description": "initial_group"}), 
              on="initial_group", how="left")

In [13]:
df.drop(labels=["initial_group", "clearance_group", "date", "date_at_scene"], axis=1, inplace=True)
df = df.sort_values(by="time_at_scene", axis=0)
df.reset_index(drop=True, inplace=True)

In [14]:
ini_group.to_csv("/home/nmarquez/Downloads/initial_group.csv", index=False)
cl_group.to_csv("/home/nmarquez/Downloads/clearance_group.csv", index=False)
df.to_csv("/home/nmarquez/Downloads/call_data.csv", index=False)
(df.loc[0:1,:]).to_csv("/home/nmarquez/Downloads/call_data_init.csv", index=False)

In [15]:
(df[df.time_at_scene.notnull()]).to_csv("/home/nmarquez/Downloads/call_data_full.csv", index=False)
(df[df.time_at_scene.isnull()]).to_csv("/home/nmarquez/Downloads/call_data_missing.csv", index=False)